# December 2020 - Challenge

Proposed solution by Walter Sebastian Gisler, 12/12/2020

## Problem Statement

The original problem statement can be found here: https://www.research.ibm.com/haifa/ponderthis/challenges/December2020.html

Let's consider a simplified version of the USA elector-based election system. In our system, the states are represented by a vector pop = [p1, p2, ..., pn] of odd numbers, given the number of eligible voters in any state.

Each state is allocated a given number of electors in the following manner: There are 1,001 electors in total, and they are assigned to states in proportion to their population. If p = p1+p2+...+pn, then state i receives (pi/p)*1001 electors, rounded down. The remaining electors are allocated one at a time for the states, ordered by the size of the remaining fractional elector (e.g., a state that got 3.5 electors will get another one before a state with 10.25 electors).

For example, if pop = [51, 61, 77, 89, 99] then the electors are [135, 162, 205, 236, 263].

There are two candidates in the election. If a candidate receives more than half the votes in a state, she gets all the electors for that state. The candidate with the most electors wins the election.

A vote is a vector [v1, v2,..,vn] that represents the number of votes a candidate received in all the states. The size of a vote is v1+v2+...+vn - the total number of votes for that candidate. A losing vote is a vote that results in losing the election (i.e., not getting enough electors). A losing vote is maximal with respect to a given population vector pop if any vote of a larger size is not a losing vote.

Your goal: Find a population vector p=[p1, p2, p3, p4, p5] on five states with 101<=pi<= 149 and a maximal losing vote v=[v1, v2, v3, v4, v5] such that the size of v is 71.781305% the size of p up to a millionth of a percent.

Present your answer in the following format:

[p1, p2, p3, p4, p5]

[v1, v2, v3, v4, v5]

## Solution

First of all, let's define a method that calculates the number of electors for a given population vector p. I think this got a little bit inefficient due to the sorting, but luckily it isn't a hard problem, so efficiency is not super important.

In [1]:
def calculate_electors(p, num_electors = 1001):
    tp = sum(p)
    fractional_e = [pi/tp*num_electors for pi in p]
    e = [int(fe) for fe in fractional_e]
    remaining = num_electors - sum(e)
    remainder = [fe-e[i] for i,fe in enumerate(fractional_e)]
    top_remainders = sorted(remainder[:])[-remaining:]
    for i,ee in enumerate(e):
        if remainder[i] in top_remainders:
            e[i] += 1
            top_remainders.remove(remainder[i])
    return e

Next, I defined a method to calculate a maximal losing vote. I started by formulating this as a MIP. Obviously, this isn't the most efficient way of doing this, but I figured it wouldn't be bad to have a MIP to be able to check the implementation of a more efficient implementation.

My second implementation is working as follows: we assume that every possible candidate for a maximal losing vote has 100% votes in some state and is lacking exactly one vote to win all the other states. We now just need to check every subset of populations. We need to check whether this subset would form a losing vote (i.e. if I add up the population in this subset, it has to be smaller than 1001/2. Since we have only 5 states, this can be done efficiently. There are:

ncr(5,1) + ncr(5,2) + ncr(5,3) + ncr(5,4) = 5 + 10 + 10 + 5

= 20 possible subsets to check. If we had more states, this would very quickly get much bigger numbers, and this method wouldn't work anymore. However, this is essentially a knapsack problem, so there we could still solve this somewhat efficiently using dynamic programming - just in case ;)

In [2]:
from docplex.mp.model import Model
from itertools import combinations

def find_maximal_losing_vote_mip(p, num_electors = 1001):
    e = calculate_electors(p, num_electors)
    model = Model()
    v = {i:model.integer_var(0,pp) for i,pp in enumerate(p)}
    win = {i:model.binary_var() for i,pp in enumerate(p)}
    elector_votes = 0
    for i,pp in enumerate(p):
        model.add(win[i]*pp >= v[i] - int(pp/2))
        model.add(win[i]*(int(pp/2)) <= v[i])
        elector_votes += win[i]*e[i]
    model.add(elector_votes <= int(num_electors/2))
    model.maximize(model.sum(v.values()))
    model.solve(log_output = False)
    solution = []
    for i,vv in v.items():
        solution.append(int(vv.solution_value))
    return solution

def find_maximal_losing_vote(p, num_electors = 1001):
    ii = list(range(len(p)))
    e = calculate_electors(p, num_electors)
    v_max = 0
    best_solution = []
    for i in ii[1:]:
        for comb in combinations(ii, i):
            losing = sum([e[j] for j in comb]) <= int(num_electors)/2
            solution = [p[j] if j in comb else int(p[j]/2) for j in ii]
            v = sum(solution)
            if losing and v > v_max:
                v_max = v
                best_solution = solution[:]
    return best_solution

Let's write quickly check whether the electors are calculated correctly for sample population:

In [3]:
calculate_electors([51, 61, 77, 89, 99])

[135, 162, 205, 236, 263]

And let's check what the maximal losing vote is for the given sample:

In [4]:
find_maximal_losing_vote([51, 61, 77, 89, 99])

[25, 30, 38, 89, 99]

Looks correct. Ok, now we just need to check all possible populations where each component of the population vector is between 101 and 149 and an odd number. We can do that with 5 nested loops. Not nice, but with only 5 states it is not worth writing something more efficient. An alternative would be itertools.product, but the disadvantage with this is that we would have permutations of the same population vector, which would be inefficient. With 5 loops we can assure that we are ignoring permutations:

In [5]:
from time import time

target_value = 0.71781305
precision = 0.000001

counter = 0
solution_counter = 0
start_time = time()
for p1 in range(101,150,2):
    for p2 in range(p1,150,2):
        for p3 in range(p2,150,2):
            for p4 in range(p3,150,2):
                for p5 in range(p4,150,2):
                    counter += 1
                    p = [p1,p2,p3,p4,p5]
                    a = calculate_electors(p)
                    v_max = find_maximal_losing_vote(p)
                    if abs(sum(v_max)/sum(p) - target_value) <= precision:
                        solution_counter += 1
                        print(p)
                        print(v_max)
                        print(sum(v_max)/sum(p))
                        print('-----')
print('\nPopulations tested: %i, Solutions found: %i, Time elapsed: %f seconds'%(counter, solution_counter, time()-start_time))

[101, 101, 115, 115, 135]
[50, 50, 115, 57, 135]
0.7178130511463845
-----
[101, 101, 115, 117, 133]
[50, 50, 57, 117, 133]
0.7178130511463845
-----
[101, 101, 115, 119, 131]
[50, 50, 57, 119, 131]
0.7178130511463845
-----
[101, 101, 115, 121, 129]
[50, 50, 57, 121, 129]
0.7178130511463845
-----
[101, 101, 115, 123, 127]
[50, 50, 57, 123, 127]
0.7178130511463845
-----
[101, 101, 115, 125, 125]
[50, 50, 57, 125, 125]
0.7178130511463845
-----
[101, 103, 113, 113, 137]
[50, 51, 113, 56, 137]
0.7178130511463845
-----
[101, 103, 113, 115, 135]
[50, 51, 56, 115, 135]
0.7178130511463845
-----
[101, 103, 113, 117, 133]
[50, 51, 56, 117, 133]
0.7178130511463845
-----
[101, 103, 113, 119, 131]
[50, 51, 56, 119, 131]
0.7178130511463845
-----
[101, 103, 113, 121, 129]
[50, 51, 56, 121, 129]
0.7178130511463845
-----
[101, 103, 113, 123, 127]
[50, 51, 56, 123, 127]
0.7178130511463845
-----
[101, 103, 113, 125, 125]
[50, 51, 56, 125, 125]
0.7178130511463845
-----
[101, 105, 111, 111, 139]
[50, 52, 111

Great. We found several solutions to the problem. I'll submit one where every single population is different. The sample solution I am submitting is the following:

[103, 105, 109, 123, 127]

[51, 52, 54, 123, 127]

We can do a quick check to see if we are indeed losing with this solution and that the size of v is 71.781305% the size of p up to a millionth of a percent:

In [6]:
p = [103, 105, 109, 123, 127]
v = [51,52,54,123,127]

print('Is the solution we found equivalent to the solution we find with the MIP?')
v_mip = find_maximal_losing_vote_mip(p)
print(v_mip == v)
print()

# Let's check how many elector votes the candidate gets in this case and if this is indeed making him lose the election:
e = calculate_electors(p)
print('Each state has the following number of electors:')
print(e)
print('The following number of electors are voting for our candidate: ')
elector_votes = [e[i] if e[i]/2 < v[i] else 0 for i,vv in enumerate(v)]
print(elector_votes)
total_elector_votes = sum(elector_votes)
print('Total number of elector votes:')
print(total_elector_votes)
print('Which is not enough to win the election:')
print(total_elector_votes < 1001/2)
print()

print('The size of v in percentages of the size of p:')
print('%f%%'%(sum(v)/sum(p)*100))

Is the solution we found equivalent to the solution we find with the MIP?
True

Each state has the following number of electors:
[182, 185, 193, 217, 224]
The following number of electors are voting for our candidate: 
[0, 0, 0, 217, 224]
Total number of elector votes:
441
Which is not enough to win the election:
True

The size of v in percentages of the size of p:
71.781305%


All looks good, the solution is obviously correct. That was a simple one. Probably the easiest Ponder Challenge I have solved so far.